# Imports and Configurations

In [1]:
import numpy as np
#from mnist import MNIST
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sb
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC, LinearSVC
import warnings
import cv2
warnings.filterwarnings('ignore')

In [16]:
import yaml
with open("../params.yaml") as config_file:
    CONFIG = yaml.safe_load(config_file)

# Data Loading

In [3]:
def load_images(path):
    all_images = os.listdir(os.path.join(path, 'images'))
    num_images = len(all_images)
    images = []
    for image in all_images:
        images.append(
            cv2.imread(os.path.join(path, 'images', image), 
                       cv2.IMREAD_GRAYSCALE)
        )
    images = np.array(images)
    return images.reshape(num_images, 28, 28)

def load_labels(path):
    label_path = os.path.join(path, 'labels', 'labels.csv')
    return pd.read_csv(label_path)

def mean_std_normalization(data):
    normalized_data = (data - np.mean(data))/np.std(data)
    return normalized_data

def flatten_image_input(images):
    return np.reshape(images, 
                      (images.shape[0], images.shape[1]*images.shape[2]))

In [4]:
project_dir = CONFIG['base']['project_path']
train_data_path = os.path.join(project_dir, CONFIG['data_load']['train_dataset'])
test_data_path  = os.path.join(project_dir, CONFIG['data_load']['test_dataset'])

train_images, train_labels = load_images(train_data_path), load_labels(train_data_path)
test_images, test_labels = load_images(test_data_path), load_labels(test_data_path)

In [5]:
train_images = mean_std_normalization(train_images)
test_images = mean_std_normalization(test_images)

In [6]:
train_images = flatten_image_input(train_images)
test_images = flatten_image_input(test_images)

# Fit Linear SVM

In [7]:
svm = LinearSVC(dual=False)

In [8]:
svm.fit(train_images, train_labels['label'].values)

LinearSVC(dual=False)

In [21]:
save_results_path = CONFIG['svm']['output']['fit_results']

os.makedirs(os.path.join(project_dir, save_results_path), exist_ok=True)

In [22]:
np.save(os.path.join(project_dir, save_results_path, 'coeff.npy'), svm.coef_)
np.save(os.path.join(project_dir, save_results_path, 'intercept.npy'), svm.intercept_)